In [44]:
import pandas as pd
import regex as re
# from pyspark.sql import SparkSession

In [45]:
# # Créer une session Spark
# spark = SparkSession.builder.appName("DataCleaning").getOrCreate()

# # Lire le fichier CSV avec Spark
# df_spark = spark.read.csv("us_customers.csv", sep=";", header=True, encoding="utf-8")
# df_spark.show()  # Affiche le DataFrame Spark

In [46]:
df = pd.read_csv("us_customers.csv", sep=";", encoding="utf-8")
df

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Tel_opt_in,Newsletter_opt_in,Not_Useful_Column
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",Ok,No,VRAI
1,1002,Abed,Nadir,123/643/9775,93 West Main Street,No,Yes,FAUX
2,1003,Walter,/White,7066950392,298 Drugs Driveway,N,NaN,VRAI
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",Yes,Y,VRAI
4,1005,Jon,Snow,876|678|3469,123 Dragons Road,Y,No,VRAI
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,yes,Yes,VRAI
6,1007,Jeff,Winger,NaN,1209 South Street,No,No,FAUX
7,1008,Sherlock,Holmes,876|678|3469,98 Clue Drive,N,No,FAUX
8,1009,Gandalf,NaN,N/a,123 Middle Earth,Yes,NaN,FAUX
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",Yes,No,VRAI


In [47]:
# 1. Déduplication
df = df.drop_duplicates()

In [48]:
# 2. Supprimer la ou les colonnes inutiles
df = df.drop(columns=["Not_Useful_Column"])

In [49]:
# 3. Supprimer les espaces vides avant et après chaque item
df = df.apply(lambda x: x.strip() if isinstance(x, str) else x)

In [50]:
# 4. Identifier et supprimer les lignes avec cellules malformées (type object/string)
# On ne teste que les colonnes First_Name, Last_Name, Phone_Number

def is_malformed_name(cell):
    if pd.isnull(cell):
        return False
    if isinstance(cell, str):
        # Trop de ponctuation ou chiffres dans le nom/prénom
        if re.search(r"[^\w\s\-\']", cell):
            return True
        if cell.lower() in ["n/a", "na"]:
            return True
        if re.search(r"\d", cell):
            return True
    return False

def is_malformed_phone(cell):
    if pd.isnull(cell):
        return True
    digits = re.sub(r"\D", "", str(cell))
    return len(digits) != 10

malformed_mask = (
    df["First_Name"].apply(is_malformed_name) |
    df["Last_Name"].apply(is_malformed_name) |
    df["Phone_Number"].apply(lambda x: str(x).strip().lower() in ["n/a", "na"])
)
df = df[~malformed_mask]

In [51]:

# 5. Transformer la colonne Phone_Number
def clean_phone(phone):
    if pd.isnull(phone):
        return "0000000000"
    # Extraire uniquement les chiffres
    digits = re.sub(r"\D", "", str(phone))
    if len(digits) == 10:
        return f"({digits[:3]}) {digits[3:6]} - {digits[6:]}"
    else:
        return "0000000000"

df["Phone_Number"] = df["Phone_Number"].apply(clean_phone)

In [52]:

# 6. Exploser logiquement la colonne Address (si possible)
# On sépare en 'Street', 'City', 'State_Zip' si possible
address_split = df["Address"].str.split(",", n=2, expand=True)

if not address_split.empty and address_split.shape[1] > 0:
	df["Street"] = address_split[0].str.strip()
else:
	df["Street"] = None

if not address_split.empty and address_split.shape[1] > 1:
	df["City"] = address_split[1].str.strip()
else:
	df["City"] = None

if not address_split.empty and address_split.shape[1] > 2:
	df["State_Zip"] = address_split[2].str.strip()
else:
	df["State_Zip"] = None

In [53]:

# 7. Respect du RGPD sur Tel_opt_in
def normalize_optin(val, phone):
    val = str(val).strip().lower()
    if phone == "0000000000":
        return "N"
    if val in ["y", "yes", "ok"]:
        return "Y"
    return "N"

df["Tel_opt_in"] = df.apply(lambda row: normalize_optin(row["Tel_opt_in"], row["Phone_Number"]), axis=1)

In [54]:

# 8. Même logique sur Newsletter_opt_in
def normalize_newsletter(val):
    val = str(val).strip().lower()
    if val in ["y", "yes", "ok"]:
        return "Y"
    return "N"

df["Newsletter_opt_in"] = df["Newsletter_opt_in"].apply(normalize_newsletter)

# Afficher le DataFrame nettoyé
df.reset_index(drop=True, inplace=True)
df

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Tel_opt_in,Newsletter_opt_in,Street,City,State_Zip
0,1001,Frodo,Baggins,(123) 545 - 5421,"123 Shire Lane, Shire",Y,N,123 Shire Lane,Shire,None
1,1002,Abed,Nadir,(123) 643 - 9775,93 West Main Street,N,Y,93 West Main Street,None,None
2,1004,Dwight,Schrute,(123) 543 - 2345,"980 Paper Avenue, Pennsylvania, 18503",Y,Y,980 Paper Avenue,Pennsylvania,18503
3,1005,Jon,Snow,(876) 678 - 3469,123 Dragons Road,Y,N,123 Dragons Road,None,None
4,1006,Ron,Swanson,(304) 762 - 2467,768 City Parkway,Y,Y,768 City Parkway,None,None
5,1007,Jeff,Winger,0000000000,1209 South Street,N,N,1209 South Street,None,None
6,1008,Sherlock,Holmes,(876) 678 - 3469,98 Clue Drive,N,N,98 Clue Drive,None,None
7,1010,Peter,Parker,(123) 545 - 5421,"25th Main Street, New York",Y,N,25th Main Street,New York,None
8,1011,Samwise,Gamgee,0000000000,"612 Shire Lane, Shire",N,N,612 Shire Lane,Shire,None
9,1013,Don,Draper,(123) 543 - 2345,2039 Main Street,Y,N,2039 Main Street,None,None
